# Block 39: Demonstration - Part 2 (Solution)
## __Adadelta__
- A limitation of gradient descent is that it uses the same step size (learning rate) for each input variable.
- AdaGrad and RMSProp are extensions to gradient descent that add a self-adaptive learning rate for each parameter for the objective function.

- Adadelta extends AdaGrad and RMSProp by introducing consistent unit calculations for the step size, eliminating the need for an initial learning rate hyperparameter.

## Steps to Be Followed:
1. Importing the required libraries
2. Defining and plotting the objective function
3. Implementing the Adadelta algorithm
4. Optimizing the Adadelta

### Step 1: Importing the Required Libraries

- Import NumPy packages.
- Import the Python package matplotlib, which sets the padding between and around the subplots as well as the figure size.
- Import all other necessary libraries.

In [0]:
from numpy import arange
from numpy import meshgrid
from matplotlib import pyplot
from numpy import asarray
from numpy.random import rand
from numpy.random import seed
from math import sqrt


### Step 2: Defining and Plotting the Objective Function
- Define the objective function as the sum of squares of x and y.
- Set the bounds for input variables.
- Generate arrays of x and y values within the specified bounds at 0.1 increments.
- Create a mesh grid from the x and y arrays.
- Compute the objective function values for each combination of x and y in the mesh grid.
- Create a filled contour plot with 50 contour levels and 'jet' color scheme.
- Display the plot.



In [0]:
# Define the objective function to be minimized (a simple quadratic function).
def objective(x, y):
    return x**2.0 + y**2.0
# Define the search space bounds for x and y.
bounds = asarray([[-1.0, 1.0], [-1.0, 1.0]])
# Create a grid of x and y values within the specified bounds.
xaxis = arange(bounds[0,0], bounds[0,1], 0.1)
yaxis = arange(bounds[1,0], bounds[1,1], 0.1)

x, y = meshgrid(xaxis, yaxis)
# Calculate the objective function values over the grid.
results = objective(x, y)

pyplot.contourf(x, y, results, levels=50, cmap='jet')

pyplot.show()

Observation:
- The observation for the given code is that the contour plot shows a symmetric bowl-shaped pattern, indicating that the objective function has a minimum point at or near the origin (x = 0, y = 0).
- The color intensity in the plot represents the function's values, with darker regions indicating lower values and lighter regions indicating higher values.





### Step 3: Implementing Adadelta Algorithm

- Initialization and setup:

  - **solution**: Initializes the solution with random values within the given bounds.
  - **sq_grad_avg**: Keeps track of the moving average of the squared gradients.
  - **sq_para_avg**: Keeps track of the moving average of the squared parameter updates.
  - **n_iter**, **rho**, and **ep** are hyperparameters controlling the number of iterations, decay factor, and a small constant to prevent division by zero, respectively.


- Gradient Computation and Update:

  - In each iteration, the gradient of the objective function with respect to the current solution is computed.
  - For each component of the gradient, the moving average of the squared gradient is updated with the decay factor rho.
  - Then, a new solution is computed based on the ratio of the square root of the moving averages of the squared parameter updates and the squared gradients. The current gradient is also used to calculate the change in parameters.

- Build a new solution:

  - Iteratively update each variable in the solution by considering the gradient and step size specific to that variable.
  - Calculate the new position of each variable by subtracting the product of the step size and gradient from the current value and store the updated variables in a new solution.

- Evaluate the solution:

  - Evaluate the objective function for the candidate point defined by the new solution

In [0]:
# Define the derivative function for the objective function.
def derivative(x, y):
    return asarray([x * 2.0, y * 2.0])
# Implement the Adadelta optimization algorithm.
def adadelta(objective, derivative, bounds, n_iter, rho, ep=1e-3):
    solution = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
# Initialize accumulators for squared gradients and squared parameter updates.
    sq_grad_avg = [0.0 for _ in range(bounds.shape[0])]

    sq_para_avg = [0.0 for _ in range(bounds.shape[0])]

    for it in range(n_iter):
        # Calculate the gradient of the objective function.
        gradient = derivative(solution[0], solution[1])

        for i in range(gradient.shape[0]):

            sg = gradient[i]**2.0

            sq_grad_avg[i] = (sq_grad_avg[i] * rho) + (sg * (1.0-rho))

        new_solution = list()
        for i in range(solution.shape[0]):
            # Calculate the update term for the parameter.
            alpha = (ep + sqrt(sq_para_avg[i])) / (ep + sqrt(sq_grad_avg[i]))
            change = alpha * gradient[i]
            # Update the squared parameter update accumulator and the parameter value.
            sq_para_avg[i] = (sq_para_avg[i] * rho) + (change**2.0 * (1.0-rho))
            value = solution[i] - change

            new_solution.append(value)
        # Update the solution with the new parameter values.
        solution = asarray(new_solution)
        solution_eval = objective(solution[0], solution[1])

        print('>%d f(%s) = %.5f' % (it, solution, solution_eval))
    return [solution, solution_eval]

## Step 4: Optimizing the Adadelta
- Set the random seed to ensure reproducibility.
- Define the bounds for the input variables.
- Specify the number of iterations.
- Set the value of rho for the Adadelta algorithm.
- Apply the Adadelta algorithm to find the best solution and its corresponding score.
- Print messages indicating the completion of the optimization process.




In [0]:
# Set random seed and optimization parameters.
seed(1)
bounds = asarray([[-1.0, 1.0], [-1.0, 1.0]])
n_iter = 120
rho = 0.99
# Apply the Adadelta algorithm to find the best solution.
best, score = adadelta(objective, derivative, bounds, n_iter, rho)
# Print the best solution and its corresponding objective function value.
print('Done!')
print('f(%s) = %f' % (best, score))

**Observation:**
- The Adadelta algorithm is applied with 120 iterations, a momentum value of 0.99, and seed 1, resulting in a convergence to the best solution and its corresponding score.